In [1]:
import h5py
import os
from dotenv import load_dotenv
load_dotenv()


True

In [2]:
with h5py.File(os.getenv('H5PY_GLOVE_PATH'), 'r') as hdf_file:
    ground_truth = []
    query_vectors = []
    base_vectors = []
    for a in hdf_file['train']:
        base_vectors.append(list(a))
        
    # for a in hdf_file['test']:
    #     query_vectors.append(list(a))
    # for a in hdf_file['neighbors']:
    #     ground_truth.append(list(a))

In [11]:
from qdrant_client import QdrantClient
from qdrant_client.http import models
from qdrant_client.http.models import Distance, VectorParams
from qdrant_client.http.models import PointStruct
import numpy as np
import random
import pandas as pd
import time


import utils

In [21]:
qdrantClient = QdrantClient(host='localhost', port=6333, timeout=10000000)

vector_size = 25
collection_name = "ann_1M"

qdrantClient.delete_collection(collection_name=collection_name)

qdrantClient.recreate_collection(
    collection_name=collection_name,
    vectors_config=VectorParams(size=vector_size, distance=Distance.EUCLID),
)

True

In [17]:
type(base_vectors[0][0])

numpy.float32

In [22]:
base_vectors = [[float(elem) for elem in vector] for vector in base_vectors]

In [18]:
type(base_vectors_float[0][0])

float

In [23]:
batch_size = 50000
num_batches = len(base_vectors) // batch_size + int(len(base_vectors) % batch_size > 0)
print(f'Number of batches: {num_batches}')

for batch_idx in range(num_batches):
    print(f'Current progress: {(batch_idx+1)*batch_size}/{len(base_vectors)}', end='\r')
    start_idx = batch_idx * batch_size
    end_idx = min((batch_idx + 1) * batch_size, len(base_vectors))

    batch_vectors = base_vectors[start_idx:end_idx]

    qdrantClient.upsert(
        collection_name=collection_name,
        points=models.Batch(
            ids=list(range(start_idx, end_idx)),
            vectors=batch_vectors
        )
    )

Number of batches: 1
